In [ ]:
#=
Model: beam.jl

This is a 348-variable model.
=#
using Reachability, MAT, Plots

compute(o::Pair{Symbol,<:Any}...) = compute(Options(Dict{Symbol,Any}(o)))

function compute(input_options::Options)

    # =====================
    # Problem specification
    # =====================
    file = matopen("beam.mat")
    A = read(file, "A")

    # initial set
    # - x1-x300 are 0.0,
    # - the rest is in [0.002, 0.0015]
    X0 = Hyperrectangle([zeros(300); fill(0.00175, 48)], [zeros(300); fill(0.00025, 48)])

    # input set
    B = read(file, "B")
    U = B * BallInf([0.5], 0.3)

    # instantiate continuous LTI system
    S = ContinuousSystem(A, X0, U)

    # ===============
    # Problem solving
    # ===============
    if input_options[:mode] == "reach"
        problem_options = Options(:vars => [89],
                                  :partition => [(2*i-1:2*i) for i in 1:174],
                                  :plot_vars => [0, 89])

    elseif input_options[:mode] == "check"
        problem_options = Options(:vars => [89],
                                  :partition => [(2*i-1:2*i) for i in 1:174],
                                  :property => LinearConstraintProperty(sparsevec([89], [1.0], 348), 2100.)) # x89 < 2100
    end

    result = solve(S, merge(problem_options, input_options))

end # function

result = compute(:δ => 1e-3, :N => 3, :mode=>"reach", :verbosity => "info");

In [3]:
@time compute(:δ => 1e-3, :T => 20.0, :mode=>"reach", :verbosity => "info"); # benchmark settings (long)

[info] Time discretization...
[info] elapsed time: 0.69622387 seconds
[info] Reachable States Computation...
[info] - Decomposing X0
[info] elapsed time: 0.304857921 seconds
[info] - Computing successors
[info] elapsed time: 39.14303081 seconds
[info] - Total
[info] elapsed time: 39.452327539 seconds
[info] Projection...
[info] elapsed time: 0.231624087 seconds
 40.383651 seconds (74.96 M allocations: 25.261 GiB, 8.43% gc time)


In `mforets/27`:

```julia
[info] Time discretization...
[info] elapsed time: 0.85341034 seconds
[info] Reachable States Computation...
[info] - Decomposing X0
[info] elapsed time: 0.267330009 seconds
[info] - Computing successors
[info] elapsed time: 40.629711488 seconds
[info] - Total
[info] elapsed time: 40.898961112 seconds
[info] Projection...
[info] elapsed time: 0.224151163 seconds
 41.985923 seconds (74.92 M allocations: 7.216 GiB, 6.28% gc time)
```

In `master`:

```julia
[info] Time discretization...
[info] elapsed time: 0.69622387 seconds
[info] Reachable States Computation...
[info] - Decomposing X0
[info] elapsed time: 0.304857921 seconds
[info] - Computing successors
[info] elapsed time: 39.14303081 seconds
[info] - Total
[info] elapsed time: 39.452327539 seconds
[info] Projection...
[info] elapsed time: 0.231624087 seconds
 40.383651 seconds (74.96 M allocations: 25.261 GiB, 8.43% gc time)
```